<a href="https://colab.research.google.com/github/yuxuan0702/Summer-Research-Project-/blob/master/basic_data_inspect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import packages 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import difflib
import plotly.express as px

pd.options.display.max_rows = 999
plt.style.use('fivethirtyeight')
%matplotlib inline

In [0]:
#insitution information 
df = pd.read_csv('/content/drive/My Drive/Summer-Research/data/CHE.csv')
df.head()

,Institution,Control,State,Category
0,Abilene Christian University,Private,TX,"<a href=""https://www.acu.edu/coronavirus/april..."
1,Adelphi University,Private,NY,"<a href=""https://health.adelphi.edu/au_news/co..."
2,Adrian College,Private,MI,"<a href=""http://adrian.edu/news/ac-president-d..."
3,Agnes Scott College,Private,GA,"<a href=""https://www.agnesscott.edu/coronaviru..."
4,Albany College of Pharmacy and Health Sciences,Private,NY,"<a href=""https://www.acphs.edu/sites/default/f..."


In [0]:
# dataset inspect
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882 entries, 0 to 881
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Institution  882 non-null    object
 1   Control      879 non-null    object
 2   State        879 non-null    object
 3   Category     879 non-null    object
dtypes: object(4)
memory usage: 27.7+ KB


In [0]:
# inspect the rows with missing 
df[df.isnull().T.any()]

,Institution,Control,State,Category
333,"0to%20LACCD%20Community.pdf"" target=""_blank"" r...",NaN,NaN,NaN
401,"target=_blank"" rel=""nofollow noopener norefer...",NaN,NaN,NaN
409,"campus-college-experience-in-fall-2020"" target...",NaN,NaN,NaN


In [0]:
# drop the rows with missing since it doesn't have any specific meaning 
df.dropna(how='any',inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 0 to 881
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Institution  879 non-null    object
 1   Control      879 non-null    object
 2   State        879 non-null    object
 3   Category     879 non-null    object
dtypes: object(4)
memory usage: 34.3+ KB


In [0]:
#check institution duplicated 
df[df.Institution.duplicated(False)]

,Institution,Control,State,Category
146,Columbia College,Private,SC,"<a href=""https://www.columbiasc.edu/president-..."
147,Columbia College,Private,MO,"<a href=""https://www.columbiamissourian.com/ne..."
192,Duke University,Private,NC,"<a href=""https://today.duke.edu/2020/04/coordi..."
193,Duke University,Private,NC,"<a href=""https://today.duke.edu/2020/05/presid..."
205,Embry-Riddle Aeronautical University,Private,AZ,"<a href=""https://news.erau.edu/headlines/embry..."
206,Embry-Riddle Aeronautical University,Private,FL,"<a href=""https://news.erau.edu/headlines/embry..."


In [0]:
# seems two rows in duke
# so we may only keep one instituon 
df.drop(axis=0,index = 192,inplace = True)

In [0]:
# keep category 
have_decided = df[(df.Category=='Planning for online') | (df.Category=='Planning for in-person')| (df.Category=='Proposing a hybrid model')]
have_decided.rename(columns={'Category':'plan'},inplace = True)
have_decided.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Institution,Control,State,plan
74,California State University — Bakersfield,Public,CA,Planning for online
75,California State University — Channel Islands,Public,CA,Planning for online
76,California State University — Chico,Public,CA,Planning for online
77,California State University — Dominguez Hills,Public,CA,Planning for online
78,California State University — East Bay,Public,CA,Planning for online


In [0]:
#change category into plan 
hyperlink_decided = df[(df.Category!='Planning for online') & (df.Category!='Planning for in-person')& (df.Category!='Proposing a hybrid model')]
hyperlink_decided
hyperlink_decided['category'],hyperlink_decided['plan'] = hyperlink_decided['Category'].str.split('>',1).str
hyperlink_decided['plan'] = hyperlink_decided['plan'].str.replace('</a>','')
hyperlink_decided.drop(columns = ['Category','category'],inplace = True)
hyperlink_decided.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning:

Columnar iteration over characters will be deprecated in future releases.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a sl

,Institution,Control,State,plan
0,Abilene Christian University,Private,TX,Planning for in-person
1,Adelphi University,Private,NY,Proposing a hybrid model
2,Adrian College,Private,MI,Planning for in-person
3,Agnes Scott College,Private,GA,Planning for in-person
4,Albany College of Pharmacy and Health Sciences,Private,NY,Planning for in-person


In [0]:
# combine decide and hyperlink 
df = pd.concat([have_decided,hyperlink_decided])
df.shape

(878, 4)

In [0]:
# import university dataset 
university = pd.read_csv('/content/drive/My Drive/Summer-Research/data/Colleges_and_Universities.csv',usecols=['NAME','STATE','LATITUDE','LONGITUDE'])
university.head(3)

,NAME,STATE,LATITUDE,LONGITUDE
0,Shorter College,AR,34.759308,-92.260490
1,Citrus Heights Beauty College,CA,38.713353,-121.289431
2,Joe Blasco Makeup Artist Training Center,CA,34.101481,-118.287070


In [0]:
# inspect university data 
university.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7735 entries, 0 to 7734
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   NAME       7735 non-null   object 
 1   STATE      7735 non-null   object 
 2   LATITUDE   7735 non-null   float64
 3   LONGITUDE  7735 non-null   float64
dtypes: float64(2), object(2)
memory usage: 241.8+ KB


In [0]:
#define function difference 
def best_guess ( usn_name ):
  """match possible univeristies in the two dataset 

  Args: 
    usn_name : str 
    the school name you trying to guess 

  Result : str
    Most close university name 

  """
  if usn_name in list( df['Institution'] ):
      return usn_name
  else:
      close_ones = difflib.get_close_matches( usn_name, list( university['NAME'] ), 1)
      if len( close_ones ) > 0:
          return close_ones[0]
      else:
          return np.nan

In [0]:
#apply best guess 
university['instition'] = university['NAME'].apply( best_guess )
university

,NAME,STATE,LATITUDE,LONGITUDE,instition
0,Shorter College,AR,34.759308,-92.260490,Shorter College
1,Citrus Heights Beauty College,CA,38.713353,-121.289431,Citrus Heights Beauty College
2,Joe Blasco Makeup Artist Training Center,CA,34.101481,-118.287070,Joe Blasco Makeup Artist Training Center
3,Waynes College of Beauty,CA,36.700631,-121.652662,Waynes College of Beauty
4,Hult International Business School,MA,42.369930,-71.070737,Hult International Business School
...,...,...,...,...,...
7730,Connecticut Center for Massage Therapy-Groton,CT,41.347074,-72.042638,Connecticut Center for Massage Therapy-Groton
7731,Pierpont Community and Technical College,WV,39.482878,-80.160335,Pierpont Community and Technical College
7732,Universal College of Beauty Inc-Compton,CA,33.895708,-118.236350,Universal College of Beauty Inc-Compton
7733,ITT Technical Institute-Duluth,GA,34.042875,-84.176526,ITT Technical Institute-Duluth


In [0]:
# merge institution and university 
df_new = pd.merge(df,university,how='left',
                   left_on='Institution', right_on='instition')
df_new.drop(columns = ['NAME','STATE','instition'],inplace = True)
df_new.head(10)

,Institution,Control,State,plan,LATITUDE,LONGITUDE
0,California State University — Bakersfield,Public,CA,Planning for online,NaN,NaN
1,California State University — Channel Islands,Public,CA,Planning for online,NaN,NaN
2,California State University — Chico,Public,CA,Planning for online,NaN,NaN
3,California State University — Dominguez Hills,Public,CA,Planning for online,NaN,NaN
4,California State University — East Bay,Public,CA,Planning for online,NaN,NaN
5,California State University — Fresno State,Public,CA,Planning for online,NaN,NaN
6,California State University — Fullerton,Public,CA,Planning for online,NaN,NaN
7,California State University — Humboldt,Public,CA,Planning for online,NaN,NaN
8,California State University — Long Beach,Public,CA,Planning for online,NaN,NaN
9,California State University — Los Angeles,Public,CA,Planning for online,NaN,NaN


In [0]:
df_new.to_csv('new.csv')